# Orderbook with Heaps - Complete Solution

## Efficient Implementation
Uses heaps for O(log P) best price lookups with negative prices for max heap behavior.

## Setup

In [ ]:
from collections import deque
import heapq
from enum import Enum
from typing import List, Tuple, Optional, Dict

class Side(Enum):
    BUY = "BUY"
    SELL = "SELL"

class OrderStatus(Enum):
    OPEN = "OPEN"
    PARTIALLY_FILLED = "PARTIALLY_FILLED"
    FILLED = "FILLED"
    CANCELLED = "CANCELLED"

class Order:
    def __init__(self, order_id: str, side: Side, price: float, quantity: int):
        self.order_id = order_id
        self.side = side
        self.price = price
        self.quantity = quantity
        self.filled_quantity = 0
        self.status = OrderStatus.OPEN
    
    def remaining_quantity(self) -> int:
        return self.quantity - self.filled_quantity
    
    def __repr__(self):
        return f"Order({self.order_id}, {self.side.value}, ${self.price}, qty={self.quantity}, filled={self.filled_quantity}, {self.status.value})"

## OrderBook Class

In [ ]:
class OrderBook:
    def __init__(self):
        # Price level: price → deque of orders
        self.bids: Dict[float, deque] = {}
        self.asks: Dict[float, deque] = {}
        
        # Heaps for fast best price lookup
        self.bid_heap: List[float] = []  # Store negative prices!
        self.ask_heap: List[float] = []  # Store positive prices
        
        # Order lookup
        self.orders: Dict[str, Order] = {}

## Solution 1: Submit Order

In [ ]:
def submit_order(self, order_id: str, side: Side, price: float, quantity: int) -> bool:
    """Submit order - O(log P)."""
    # Check if order already exists
    if order_id in self.orders:
        return False
    
    # Create order
    order = Order(order_id, side, price, quantity)
    self.orders[order_id] = order
    
    if side == Side.BUY:
        # Add to bids
        if price not in self.bids:
            self.bids[price] = deque()
            heapq.heappush(self.bid_heap, -price)  # NEGATIVE for max heap!
        self.bids[price].append(order)
    else:
        # Add to asks
        if price not in self.asks:
            self.asks[price] = deque()
            heapq.heappush(self.ask_heap, price)  # Positive for min heap
        self.asks[price].append(order)
    
    return True

OrderBook.submit_order = submit_order

## Solution 2: Cancel Order

In [ ]:
def cancel_order(self, order_id: str) -> bool:
    """Cancel order - O(M) where M is orders at price level."""
    # Look up order
    if order_id not in self.orders:
        return False
    
    order = self.orders[order_id]
    
    # Check if can be cancelled
    if order.status in [OrderStatus.FILLED, OrderStatus.CANCELLED]:
        return False
    
    # Mark as cancelled
    order.status = OrderStatus.CANCELLED
    
    # Remove from price level queue
    price_levels = self.bids if order.side == Side.BUY else self.asks
    
    if order.price in price_levels:
        queue = price_levels[order.price]
        
        try:
            queue.remove(order)
            
            # Clean up empty price level
            if len(queue) == 0:
                del price_levels[order.price]
        except ValueError:
            pass  # Order not in queue
    
    return True

OrderBook.cancel_order = cancel_order

## Get Order Status

In [ ]:
def get_order_status(self, order_id: str) -> Optional[Order]:
    """Look up order by ID - O(1)."""
    return self.orders.get(order_id)

OrderBook.get_order_status = get_order_status

## Solution 3: Get Best Bid

In [ ]:
def get_best_bid(self) -> Optional[float]:
    """Get highest bid price - O(1) amortized."""
    while self.bid_heap:
        # Get price from heap (negate to get actual price)
        price = -self.bid_heap[0]
        
        # Check if price is valid (has orders)
        if price in self.bids and len(self.bids[price]) > 0:
            return price
        
        # Remove stale price
        heapq.heappop(self.bid_heap)
        if price in self.bids:
            del self.bids[price]
    
    return None

OrderBook.get_best_bid = get_best_bid

## Solution 4: Get Best Ask

In [ ]:
def get_best_ask(self) -> Optional[float]:
    """Get lowest ask price - O(1) amortized."""
    while self.ask_heap:
        # Get price from heap (already positive)
        price = self.ask_heap[0]
        
        # Check if price is valid (has orders)
        if price in self.asks and len(self.asks[price]) > 0:
            return price
        
        # Remove stale price
        heapq.heappop(self.ask_heap)
        if price in self.asks:
            del self.asks[price]
    
    return None

OrderBook.get_best_ask = get_best_ask

## Solution 5: Match Order

In [ ]:
def match_order(self, order_id: str, side: Side, price: float, quantity: int) -> List[Tuple[str, str, float, int]]:
    """Match order against opposite side - O(M) per price level."""
    trades = []
    
    # Submit order
    if not self.submit_order(order_id, side, price, quantity):
        return trades
    
    incoming = self.orders[order_id]
    
    if side == Side.BUY:
        # BUY order: match against asks
        best_ask = self.get_best_ask()
        if best_ask is None or best_ask > price:
            return trades  # No match
        
        ask_orders = self.asks[best_ask]
        
        for resting in list(ask_orders):  # Use list() to avoid modification issues
            if resting.status != OrderStatus.OPEN:
                continue
            
            # Calculate trade quantity
            trade_qty = min(incoming.remaining_quantity(), resting.remaining_quantity())
            
            # Update quantities
            incoming.filled_quantity += trade_qty
            resting.filled_quantity += trade_qty
            
            # Update statuses
            if incoming.remaining_quantity() == 0:
                incoming.status = OrderStatus.FILLED
            elif incoming.filled_quantity > 0:
                incoming.status = OrderStatus.PARTIALLY_FILLED
                
            if resting.remaining_quantity() == 0:
                resting.status = OrderStatus.FILLED
            elif resting.filled_quantity > 0:
                resting.status = OrderStatus.PARTIALLY_FILLED
            
            # Record trade (buyer, seller, price, qty)
            trades.append((order_id, resting.order_id, best_ask, trade_qty))
            
            if incoming.remaining_quantity() == 0:
                break
    
    else:  # Side.SELL
        # SELL order: match against bids
        best_bid = self.get_best_bid()
        if best_bid is None or best_bid < price:
            return trades  # No match
        
        bid_orders = self.bids[best_bid]
        
        for resting in list(bid_orders):
            if resting.status != OrderStatus.OPEN:
                continue
            
            # Calculate trade quantity
            trade_qty = min(incoming.remaining_quantity(), resting.remaining_quantity())
            
            # Update quantities
            incoming.filled_quantity += trade_qty
            resting.filled_quantity += trade_qty
            
            # Update statuses
            if incoming.remaining_quantity() == 0:
                incoming.status = OrderStatus.FILLED
            elif incoming.filled_quantity > 0:
                incoming.status = OrderStatus.PARTIALLY_FILLED
                
            if resting.remaining_quantity() == 0:
                resting.status = OrderStatus.FILLED
            elif resting.filled_quantity > 0:
                resting.status = OrderStatus.PARTIALLY_FILLED
            
            # Record trade (buyer, seller, price, qty)
            # Note: resting is buyer, incoming is seller!
            trades.append((resting.order_id, order_id, best_bid, trade_qty))
            
            if incoming.remaining_quantity() == 0:
                break
    
    return trades

OrderBook.match_order = match_order

## Pretty Print

In [ ]:
def __repr__(self):
    lines = ["=" * 60, "ORDERBOOK", "=" * 60]
    
    # Show asks (high to low)
    for price in sorted(self.asks.keys(), reverse=True):
        total = sum(o.remaining_quantity() for o in self.asks[price] if o.status == OrderStatus.OPEN)
        if total > 0:
            lines.append(f"ASK: ${price:>8.2f} | Qty: {total}")
    
    lines.append("-" * 60)
    
    # Show bids (high to low)
    for price in sorted(self.bids.keys(), reverse=True):
        total = sum(o.remaining_quantity() for o in self.bids[price] if o.status == OrderStatus.OPEN)
        if total > 0:
            lines.append(f"BID: ${price:>8.2f} | Qty: {total}")
    
    lines.append("=" * 60)
    best_bid = self.get_best_bid()
    best_ask = self.get_best_ask()
    if best_bid and best_ask:
        lines.append(f"Spread: ${best_ask - best_bid:.2f}")
    return "\n".join(lines)

OrderBook.__repr__ = __repr__

## Test Suite

In [ ]:
# Test 1: Submit orders
print("Test 1: Submit orders")
ob = OrderBook()

ob.submit_order("B1", Side.BUY, 100.0, 50)
ob.submit_order("B2", Side.BUY, 99.0, 30)
ob.submit_order("A1", Side.SELL, 101.0, 40)
ob.submit_order("A2", Side.SELL, 102.0, 25)

print(f"Best Bid: ${ob.get_best_bid()}")
print(f"Best Ask: ${ob.get_best_ask()}")
print(ob)

In [ ]:
# Test 2: Order status
print("Test 2: Order status")
order = ob.get_order_status("B1")
print(order)

In [ ]:
# Test 3: Cancel order
print("Test 3: Cancel order B2")
ob.cancel_order("B2")
print(f"Best Bid after cancel: ${ob.get_best_bid()}")
print(ob)

In [ ]:
# Test 4: Match sell order
print("Test 4: Match sell order at $99")
trades = ob.match_order("S1", Side.SELL, 99.0, 30)
print(f"Trades: {len(trades)}")
for buyer, seller, price, qty in trades:
    print(f"  {buyer} bought {qty} from {seller} at ${price}")
print(ob)

In [ ]:
# Test 5: Match buy order
print("Test 5: Match buy order at $105")
trades = ob.match_order("B3", Side.BUY, 105.0, 60)
print(f"Trades: {len(trades)}")
for buyer, seller, price, qty in trades:
    print(f"  {buyer} bought {qty} from {seller} at ${price}")
print(ob)

In [ ]:
# Test 6: Final statuses
print("\nFinal order statuses:")
for order_id in ["B1", "S1", "A1", "B3"]:
    order = ob.get_order_status(order_id)
    if order:
        print(f"  {order}")

## Time Complexity Summary

| Operation | Time Complexity | Explanation |
|-----------|----------------|-------------|
| Submit | O(log P) | Heap push for new price level |
| Cancel | O(M) | Remove from deque (scan required) |
| Get Status | O(1) | Dict lookup |
| Get Best Bid/Ask | O(1) amortized | Heap top with lazy cleanup |
| Match | O(M) | Iterate orders at price level |

Where:
- P = number of unique price levels
- M = orders at a price level (typically small)